In [66]:
import numpy as np
import torch
import torch.nn as nn
import pickle
from torch.utils.data import DataLoader, TensorDataset

device = (
    "cuda"
    if torch.cuda.is_available()
    else "cpu"
)
 

Using cpu


In [69]:
def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.stack = nn.Sequential(
            nn.Linear(32*32*3, 32*32),
            nn.ReLU(),
            nn.Linear(32*32, 32*32),
            nn.ReLU(),
            nn.Linear(32*32, 10)
        )
    def forward(self, x):
        logits = self.stack(x)
        return logits

In [79]:
classes = unpickle('./cifar/batches.meta')[b'label_names']
labels = []
data = []
test_labels = []
test_data = []
for i in range(1, 6):
    path = "./cifar/data_batch_" + str(i)
    labels += (unpickle(path)[b'labels'])
    for j in unpickle(path)[b'data']:
        data.append(j)
test_labels += unpickle("./cifar/test_batch")[b'labels']
for i in unpickle("./cifar/test_batch")[b'data']:
    test_data.append(i)
tensor_X = torch.Tensor(data)
tensor_y = torch.Tensor(labels)

torch.Size([50000, 3072])


In [81]:
train_dataset = TensorDataset(tensor_X, tensor_y)
train_loader = DataLoader(train_dataset, batch_size=32)
#having initialised dataset and dataloader

#initialise model, loss fn., optimiser
model = Classifier()
loss = nn.CrossEntropyLoss()
optimiser = torch.optim.Adam(model.parameters())
print(model)

Classifier(
  (stack): Sequential(
    (0): Linear(in_features=3072, out_features=1024, bias=True)
    (1): ReLU()
    (2): Linear(in_features=1024, out_features=1024, bias=True)
    (3): ReLU()
    (4): Linear(in_features=1024, out_features=10, bias=True)
  )
)
